In [10]:
# to access the OS disk
import os

# deal with URL images
import urllib

# for data structure
import pandas as pd


In [20]:
cam_names = {'Cuyamaca_Peak',
              'California_School',
              'Tenerife_Observatory',
              'Kelowna_Airport',
              'Phoenix_Arizona',
             }

metadata = pd.DataFrame(index=cam_names, columns=('lat', 'long', 'time_zone', 'webcam_url'))

metadata.loc['Cuyamaca_Peak'] = (32.916783, -116.631822,'UTC-08:00','http://www.creekbed.org/cuyacam_000.jpg')
metadata.loc['California_School'] = (32.668000, -116.295743, 'UTC-08:00', 'http://wwc.instacam.com/instacamimg/BLVRD/BLVRD_l.jpg')
metadata.loc['Tenerife_Observatory'] = (28.302559, -16.510025, 'UTC+00:00', 'http://www.telescope.org/v4webcams-i.php?cam=teide')
metadata.loc['Kelowna_Airport'] = (49.957878, -119.378719, 'UTC-08:00', 'http://www.metcam.navcanada.ca/dawc_images/wxcam/CYLW/CYLW_N-full-e.jpeg')
metadata.loc['Phoenix_Arizona'] = (33.583734, -112.072115, 'UTC-07:00', 'http://www.nps.gov/webcams-yell/mammoth3.jpg')
metadata

,lat,long,time_zone,webcam_url
California_School,32.668,-116.2957,UTC-08:00,http://wwc.instacam.com/instacamimg/BLVRD/BLVR...
Cuyamaca_Peak,32.91678,-116.6318,UTC-08:00,http://www.creekbed.org/cuyacam_000.jpg
Tenerife_Observatory,28.30256,-16.51002,UTC+00:00,http://www.telescope.org/v4webcams-i.php?cam=t...
Kelowna_Airport,49.95788,-119.3787,UTC-08:00,http://www.metcam.navcanada.ca/dawc_images/wxc...
Phoenix_Arizona,33.58373,-112.0721,UTC-07:00,http://www.nps.gov/webcams-yell/mammoth3.jpg


In [9]:
# check if folders exist
for name in webcam_url:
    path = './' + name
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
urllib.urlretrieve("http://www.digimouth.com/news/media/2011/09/google-logo.jpg", "local-filename.jpg")